In [1]:
from __future__ import print_function, division
from scipy.optimize import fsolve #to get the initial value of vecp0
import numpy as np
from scikits.odes import dae
import pandas as pd
import time
import cantera as ct
import os
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import prettyplotlib as ppl
from prettyplotlib import brewer2mpl
from scipy.optimize import fsolve, root
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['font.family'] = 'Times New Roman'
colors = brewer2mpl.get_map('Set2','qualitative',8).mpl_colors
from decimal import *
getcontext().prec = 40 #start a new precision
from scikits.odes.odeint import odeint
from scikits.odes import ode

In [2]:
###########################CVOD for outer function##########################################################
def RhsEq(z, vec, vecp):
    """ vec = [u, rho, p]"""
    mech = '/Users/yuanjie/Dropbox/Cantera_intern/code/H2O_absorp/small_mec.cti'
    gas = ct.Solution(mech,'gas')
    surf_phase = ct.Interface(mech, 'oxide_surface',[gas])
    T = 500 #K
    R =  8314 #universal gas constant
    D = 5.08 * 10**-2 #diameter of the tube [m]
    Ac = np.pi * D**2/4 # cross section of the tube [m]
    nu = 1.418E-7 #m2/s #kinetic viscosity of the water vapor
    perim = np.pi * D #perimeter of the tube
    sigma_k = [0,2] 

    #gas.set_unnormalized_mass_fractions(vec[2:2+gas.n_species])
    #gas.TPY = T,vec[2+gas.n_species],'H2O:%f'%(vec[2:2+gas.n_species])
    gas.TPY = T,vec[2],'H2O:1'
    surf_phase.set_unnormalized_coverages
    surf_phase.TP = T,vec[2]
    sigma_k = [0,2] 
    ################################# inner IDA solver ###################################################
    """use the ida solver to solve the trasient site fraction equation to get the site fraction of surface species at steady state"""
    Z0 = surf_phase.coverages
    Zp0 = surf_phase.net_production_rates[-surf_phase.n_species:]*sigma_k[:surf_phase.n_species]/surf_phase.site_density
    def residual_dZkdt(t, Z, dotZ, result):
        """Z is a vector stored the site fraction of all surface species"""
        surf_phase.coverages = Z
        for i in range(surf_phase.n_species):
            result[i] = dotZ[i] - surf_phase.net_production_rates[-surf_phase.n_species+i]*sigma_k[i]/surf_phase.site_density
        np.delete(result, np.argmax(Z0)) # remove the largest site fraction element
        result[np.argmax(Z0)] = np.sum(Z) - 1
        
    solver = dae('ida', residual_dZkdt, 
                 compute_initcond='yp0', #If yp0, then the differential variables (y of the ode system at time 0) will be used to solve for the derivatives of the differential variables, so yp0 will be calculated
                 first_step_size=1e-18,
                 atol=1e-6, #absolute tolerance for solution
                 rtol=1e-6, #relative tolerance for solution
                 algebraic_vars_idx = [np.argmax(Z0)], #If the given problem is of type DAE, some items of the residual
                 max_steps = 5000,
                 old_api = False)#Forces use of old api (tuple of 7) if True or


    t = 0
    index = range(surf_phase.n_species)
    np.delete(index,np.argmax(Z0))
    solution = solver.solve(np.array([0,100]), Z0, Zp0) #assume that when t = 100s, the system reaches the steady state
    surf_phase.coverages = solution.values.y[1,:]
    """
    while np.sum(np.abs(surf_phase.net_production_rates[-surf_phase.n_species:][index] - np.zeros(surf_phase.n_species-1)))>1e-3:
        t = t + 5
        solution = solver.solve(np.array([0,t]), Z0, Zp0)
    """
    
    ############################################# reach steady state #########################################
    sdot_gas = surf_phase.net_production_rates[:gas.n_species]
    ##equation y' = f(t,y)
    #vec = [u, rho, p]
    vecp[0] = (perim/Ac*np.sum(sdot_gas*gas.molecular_weights) - vec[0]*vecp[1])/vec[1]
    vecp[1] = 1/(R*T)*(gas.mean_molecular_weight*vecp[2])
    vecp[2] = -2*vec[1]*vec[0]*vecp[0]- np.power(vec[0],2)*vecp[1] - 32*vec[1]*vec[0]*nu/D**2  
solver = ode('cvode', RhsEq, old_api=False)
result = solver.solve([0., 0.2], [11.53, 0.24393884402135085, 1.01325000e+05])

In [3]:
result

SolverReturn(flag=<StatusEnum.SUCCESS: 0>, values=SolverVariables(t=array([0. , 0.2]), y=array([[1.15300000e+01, 2.43938844e-01, 1.01325000e+05],
       [1.15300002e+01, 2.43938840e-01, 1.01324999e+05]])), errors=SolverVariables(t=None, y=None), roots=SolverVariables(t=None, y=None), tstop=SolverVariables(t=None, y=None), message='Successful function return.')